# Imports
Importing necessary packages

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Model Architecture
Designing the model structure

In [0]:
class Net(nn.Module):
    def __init__(self):
        """ This function instantiates all the model layers """
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, 3)  # Input: 28x28x1 | Output: 26x26x16 | RF: 3x3
        self.norm1 = nn.BatchNorm2d(16)
        self.dropout1 = nn.Dropout(0.15)

        self.conv2 = nn.Conv2d(16, 16, 3)  # Input: 26x26x16 | Output: 24x24x16 | RF: 5x5
        self.norm2 = nn.BatchNorm2d(16)
        self.dropout2 = nn.Dropout(0.15)

        self.conv3 = nn.Conv2d(16, 16, 3)  # Input: 24x24x16 | Output: 22x22x32 | RF: 7x7
        self.norm3 = nn.BatchNorm2d(16)

        self.pool = nn.MaxPool2d(2, 2)  # Input: 22x22x32 | Output: 11x11x32 | RF: 14x14

        self.conv4 = nn.Conv2d(16, 16, 3)  # Input: 9x9x16 | Output: 7x7x16 | RF: 18x18
        self.norm4 = nn.BatchNorm2d(16)
        self.dropout4 = nn.Dropout(0.15)

        self.conv5 = nn.Conv2d(16, 16, 3)  # Input: 7x7x16 | Output: 5x5x16 | RF: 20x20
        self.norm5 = nn.BatchNorm2d(16)
        self.dropout5 = nn.Dropout(0.15)

        self.conv6 = nn.Conv2d(16, 10, 3)  # Input: 5x5x16 | Output: 3x3x10 | RF: 22x22

        self.gap = nn.AdaptiveAvgPool2d(1)  # Input: 3x3x10 | Output: 1x1x10
    
    def forward(self, x):
        """ This function defines the network structure """
        net = self.dropout1(self.norm1(F.relu(self.conv1(x))))
        net = self.dropout2(self.norm2(F.relu(self.conv2(net))))
        net = self.pool(self.norm3(F.relu(self.conv3(net))))
        net = self.dropout4(self.norm4(F.relu(self.conv4(net))))
        net = self.dropout5(self.norm5(F.relu(self.conv5(net))))
        net = F.relu(self.conv6(net))
        net = self.gap(net)
        net = net.view(-1, 10)
        return F.log_softmax(net)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 16, 22, 22]           2,320
       BatchNorm2d-8           [-1, 16, 22, 22]              32
         MaxPool2d-9           [-1, 16, 11, 11]               0
           Conv2d-10             [-1, 16, 9, 9]           2,320
      BatchNorm2d-11             [-1, 16, 9, 9]              32
          Dropout-12             [-1, 16, 9, 9]               0
           Conv2d-13             [-1, 16, 7, 7]           2,320
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


# Dataset Preparation
- Downloading the MNIST dataset
- Normalizing the dataset
- Splitting the dataset into batches
- Creating dataset generator for train and test set

In [0]:
torch.manual_seed(1)

batch_size = 32

mean = 0.1307
std = 0.3081
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        '../data', train=True, download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((mean,), (std,))
        ])
    ), batch_size=batch_size, shuffle=True, **kwargs
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        '../data', train=False,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((mean,), (std,))
        ])
    ), batch_size=batch_size, shuffle=True, **kwargs
)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


# Model Training and Evaluation
Training the model and testing it on the test dataset

In [0]:
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print(f'Epoch {epoch}:')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 1:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0481, Accuracy: 9858/10000 (98.58%)

Epoch 2:


  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0178, Accuracy: 9947/10000 (99.47%)

Epoch 18:


  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0180, Accuracy: 9947/10000 (99.47%)

Epoch 19:


loss=0.006630152463912964 batch_id=1874: 100%|██████████| 1875/1875 [00:22<00:00, 82.64it/s]



Test set: Average loss: 0.0168, Accuracy: 9949/10000 (99.49%)

